In [1]:
import torch
import torch.nn as nn
from einops import rearrange, repeat

### Patch Embedding
It has three components:
- Convert the image into sequence of patches
- Add CLS token to sequence of patches
- Add positional encoding to all the patches. 

In [19]:
class PatchEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        image_height = config["image_height"]
        image_width = config["image_width"]
        im_channels = config["im_channels"]
        emb_dim = config["emb_dim"] # Transformer dimentions(D)
        patch_embd_dropout = config["patch_emb_drop"]

        self.patch_height = config["patch_height"]
        self.patch_width = config["patch_width"]

        num_patches = (image_height // self.patch_height) * (image_width // self.patch_width)

        patch_dim = im_channels * self.patch_height * self.patch_width    
        
        # W belongs to R^(patch_dim x emb_dim)
        self.patch_emb = nn.Sequential(
            nn.LayerNorm(patch_dim),           
            nn.Linear(patch_dim, emb_dim),
            nn.LayerNorm(emb_dim),
        )
        
        # Positional information needs to be added to cls as well so 1+num_patches
        self.pos_emb = nn.Parameter(torch.randn(1, num_patches + 1, emb_dim))
        self.cls_token = nn.Parameter(torch.randn(emb_dim))    # CLS token belongs to R^emb_dim
        self.patch_emb_dropout = nn.Dropout(patch_embd_dropout)

    def forward(self, x):
        batch_size = x.shape[0]

        out  = rearrange(x, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=self.patch_height, p2=self.patch_width)  # split image into patches

        out = self.patch_emb(out)
        cls_token = repeat(self.cls_token, 'd -> b n d', b=batch_size, n=1)
        out = torch.cat([cls_token, out], dim=1)
        out += self.pos_emb
        out = self.patch_emb_dropout(out)

        return out


In [51]:
#example run
image = torch.randn(1, 3, 224, 224)
config = {
    "image_height": 224,
    "image_width": 224,
    "im_channels": 3,
    "emb_dim": 512,
    "patch_height": 16,
    "patch_width": 16,
    "patch_emb_dropout": 0.1
}
patch_emb = PatchEmbedding(config)
out = patch_emb(image)
print(out.shape)  

torch.Size([1, 197, 512])


### Attention Module

In [3]:
class Attention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.n_heads = config["n_heads"]
        self.head_dim = config["head_dim"]   # d_h
        self.emb_dim = config["emb_dim"]
        self.drop_prob = config["dropout"] if "dropout" in config else 0.0
        self.att_dim  = self.n_heads * self.head_dim

        self.qkv_proj = nn.Linear(self.emb_dim, self.att_dim * 3, bias=False)
        self.att_drop = nn.Dropout(self.drop_prob)

        self.out_proj = nn.Sequential(
            nn.Linear(self.att_dim, self.emb_dim),
            nn.Dropout(self.drop_prob)
        )

    def forward(self, x):
        B, N = x.shape[:2]  # B: batch size, N: number of tokens

        q, k,v = self.qkv_proj(x).split(self.att_dim, dim=-1)
        #split into heads
        q = rearrange(q, 'b n (h d_h) -> b h n d_h', h=self.n_heads, d_h=self.head_dim) 
        k = rearrange(k, 'b n (h d_h) -> b h n d_h', h=self.n_heads, d_h=self.head_dim)
        v = rearrange(v, 'b n (h d_h) -> b h n d_h', h=self.n_heads, d_h=self.head_dim)

        #Scaled dot product attention

        att = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)
        att = nn.functional.softmax(att, dim=-1)
        att = self.att_drop(att)


        #Weighted Value Computation
        out = torch.matmul(att, v)

        #Rearrange heads
        out = rearrange(out, 'b h n d_h -> b n (h d_h)', h=self.n_heads, d_h=self.head_dim)
        out = self.out_proj(out)

        return out
        

In [48]:
#example run
config = {
    "n_heads": 8,
    "head_dim": 64,
    "emb_dim": 512,
    "dropout": 0.1
}

att = Attention(config)
out = att(out)
print(out.shape)

torch.Size([1, 197, 512])


### Transformer

In [4]:
class TransformerLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        emb_dim = config["emb_dim"]
        ff_hidden_dim = config["ff_dim"] if "ff_dim" in config else 4 * emb_dim
        ff_dropout = config["ff_drop"] if "ff_drop" in config else 0.0
        self.att_norm = nn.LayerNorm(emb_dim)
        self.ff_norm = nn.LayerNorm(emb_dim)
        self.attention_block = Attention(config)
        self.ff_block = nn.Sequential(
            nn.Linear(emb_dim, ff_hidden_dim),
            nn.GELU(),
            nn.Dropout(ff_dropout),
            nn.Linear(ff_hidden_dim, emb_dim),
            nn.Dropout(ff_dropout)
        )

    def forward(self, x):
        out = x
        out = out + self.attention_block(self.att_norm(out))
        out = out + self.ff_block(self.ff_norm(out))
        return out
        

In [54]:
#expample run
config = {
    "emb_dim": 512,
    "ff_dim": 2048,
    "ff_drop": 0.1,
    "n_heads": 8,
    "head_dim": 64,
    "dropout": 0.1
}

trasnformer_block = TransformerLayer(config)
out = trasnformer_block(out)
print(out.shape)

torch.Size([1, 197, 512])


### ViT

In [15]:
class VIT(nn.Module):
    def __init__(self, config):
        super().__init__()
        n_layers = config["n_layers"]
        emb_dim = config["emb_dim"]
        num_classes = config["num_classes"] # number of classes
        self.patch_embedding = PatchEmbedding(config)
        self.transformer = nn.ModuleList([TransformerLayer(config) for _ in range(n_layers)])
        self.norm = nn.LayerNorm(emb_dim)
        self.fc_layer = nn.Linear(emb_dim, num_classes)

    def forward(self, x):
        out = self.patch_embedding(x)
        for layer in self.transformer:
            out = layer(out)
        out = self.norm(out)
        
        out = self.fc_layer(out[:, 0])

        #Logits, No softmax
        return out


In [56]:
#example run
image = torch.randn(1, 3, 224, 224)
config = {
    "image_height": 224,
    "image_width": 224,
    "im_channels": 3,
    "emb_dim": 512,
    "patch_height": 16,
    "patch_width": 16,
    "patch_emb_dropout": 0.1,
    "n_layers": 12,
    "num_dim": 1000,
    "n_heads": 8,
    "head_dim": 64,
    "dropout": 0.1
}

model = VIT(config)
out = model(image)
print(out.shape)

torch.Size([1, 1000])


#### Training

In [16]:
import sys
sys.path.append('../')
import yaml
import argparse
import torch
import random
import os
import numpy as np
from tqdm import tqdm
from torch.utils.data.dataloader import DataLoader
from dataset.mnist_color_texture_dataset import MnistDataset
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def train_for_one_epoch(epoch_idx, model, mnist_loader, optimizer):
    r"""
    Method to run the training for one epoch.
    :param epoch_idx: iteration number of current epoch
    :param model: Transformer model
    :param mnist_loader: Data loder for mnist
    :param optimizer: optimizer to be used taken from config
    :return:
    """
    losses = []
    criterion = torch.nn.CrossEntropyLoss()
    for data in tqdm(mnist_loader):
        im = data['image'].float().to(device)
        number_cls = data['number_cls'].long().to(device)
        optimizer.zero_grad()
        model_output = model(im)
        loss = criterion(model_output, number_cls)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print('Finished epoch: {} | Number Loss : {:.4f}'.
          format(epoch_idx + 1,
                 np.mean(losses)))
    return np.mean(losses)


def train(config_path):
    #  Read the config file
    ######################################
    with open(config_path, 'r') as file:
        try:
            config = yaml.safe_load(file)
        except yaml.YAMLError as exc:
            print(exc)
    print(config)
    #######################################
    
    # Set the desired seed value
    ######################################
    seed = config['train_params']['seed']
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(seed)
    #######################################
    
    # Create the model and dataset
    model = VIT(config['model_params']).to(device)
    mnist = MnistDataset('train', config['dataset_params'],
                         im_h=config['model_params']['image_height'],
                         im_w=config['model_params']['image_width'])
    mnist_loader = DataLoader(mnist, batch_size=config['train_params']['batch_size'], shuffle=True, num_workers=4)
    num_epochs = config['train_params']['epochs']
    optimizer = Adam(model.parameters(), lr=config['train_params']['lr'])
    scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2)
    
    # Create output directories
    if not os.path.exists(config['train_params']['task_name']):
        os.mkdir(config['train_params']['task_name'])
    
    # Load checkpoint if found
    if os.path.exists(os.path.join(config['train_params']['task_name'],
                                   config['train_params']['ckpt_name'])):
        print('Loading checkpoint')
        model.load_state_dict(torch.load(os.path.join(config['train_params']['task_name'],
                                                      config['train_params']['ckpt_name']), map_location=device))
    best_loss = np.inf
    
    for epoch_idx in range(num_epochs):
        mean_loss = train_for_one_epoch(epoch_idx, model, mnist_loader, optimizer)
        scheduler.step(mean_loss)
        # Simply update checkpoint if found better version
        if mean_loss < best_loss:
            print('Improved Loss to {:.4f} .... Saving Model'.format(mean_loss))
            torch.save(model.state_dict(), os.path.join(config['train_params']['task_name'],
                                                        config['train_params']['ckpt_name']))
            best_loss = mean_loss
        else:
            print('No Loss Improvement')
   

In [22]:
config_path = "../config/default.yaml"
train(config_path)

{'dataset_params': {'root_dir': '../data'}, 'model_params': {'n_heads': 8, 'head_dim': 64, 'emb_dim': 128, 'attn_drop': 0.1, 'ff_dim': 256, 'ff_drop': 0.1, 'n_layers': 6, 'bg_classes': 44, 'num_classes': 10, 'image_height': 224, 'image_width': 224, 'patch_height': 16, 'patch_width': 16, 'patch_emb_drop': 0.1, 'im_channels': 3}, 'train_params': {'task_name': 'default', 'batch_size': 64, 'epochs': 100, 'lr': 0.001, 'seed': 1111, 'ckpt_name': 'vit_ckpt.pth'}}


100%|██████████| 938/938 [01:16<00:00, 12.21it/s]


Finished epoch: 1 | Number Loss : 0.9568
Improved Loss to 0.9568 .... Saving Model


100%|██████████| 938/938 [01:18<00:00, 11.93it/s]


Finished epoch: 2 | Number Loss : 0.3287
Improved Loss to 0.3287 .... Saving Model


...

Finished epoch: 98 | Number Loss : 0.0002
No Loss Improvement


100%|██████████| 938/938 [01:16<00:00, 12.23it/s]


Finished epoch: 99 | Number Loss : 0.0002
Improved Loss to 0.0002 .... Saving Model


100%|██████████| 938/938 [01:16<00:00, 12.20it/s]

Finished epoch: 100 | Number Loss : 0.0002
No Loss Improvement
